In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp
import warnings

warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data=pd.read_csv('../input/health-insurance-cross-sell-prediction/train.csv')
test_data=pd.read_csv('../input/health-insurance-cross-sell-prediction/test.csv')

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
print('Number of Training = {}'.format(train_data.shape[0]))
print('Number of Test = {}\n'.format(test_data.shape[0]))

#target columns
train_data.columns

In [ ]:
import numpy as np
train_data["Gender"]=np.where(train_data["Gender"]=="Female",1,0)
test_data["Gender"]=np.where(test_data["Gender"]=="Female",1,0)

In [ ]:
import numpy as np
train_data["Vehicle_Damage"]=np.where(train_data["Vehicle_Damage"]=="Yes",1,0)
test_data["Vehicle_Damage"]=np.where(test_data["Vehicle_Damage"]=="Yes",1,0)

In [ ]:
train_data.head()

In [ ]:
#GET DUMMIES 
vehicle_oh= pd.get_dummies(train_data["Vehicle_Age"],prefix="vehicle_age")
gender_oh= pd.get_dummies(train_data["Gender"],prefix="gender")
dlicense_oh= pd.get_dummies(train_data["Driving_License"],prefix="driving_license")
pinsured_oh= pd.get_dummies(train_data["Previously_Insured"],prefix="previously_insured")

In [ ]:
#combine encoded

df_encoded= pd.concat([train_data,
                      vehicle_oh,
                      gender_oh,
                      dlicense_oh,
                      pinsured_oh],axis=1)

df_encoded.head()

In [ ]:
df_encoded = df_encoded.drop(["Vehicle_Age","Gender","Driving_License","Previously_Insured"],axis=1)
df_encoded.head() 

In [ ]:
#GET DUMMIES 
test_vehicle_oh= pd.get_dummies(test_data["Vehicle_Age"],prefix="vehicle_age")
test_gender_oh= pd.get_dummies(test_data["Gender"],prefix="gender")
test_dlicense_oh= pd.get_dummies(test_data["Driving_License"],prefix="driving_license")
test_pinsured_oh= pd.get_dummies(test_data["Previously_Insured"],prefix="previously_insured")



test_data= pd.concat([test_data,
                      test_vehicle_oh,
                      test_gender_oh,
                      test_dlicense_oh,
                      test_pinsured_oh],axis=1)

test_data.head()

In [ ]:
test_data = test_data.drop(["Vehicle_Age","Gender","Driving_License","Previously_Insured"],axis=1)
test_data.head() 

In [ ]:
from sklearn.model_selection import train_test_split

y= df_encoded["Response"]  #target
X = df_encoded.drop(["Response","id"], axis=1) #train predictors

X_train, X_val, y_train,y_val = train_test_split(X,y, test_size = 0.3, random_state=21, stratify=y)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

#train

logreg.fit(X_train,y_train)

#unused

y_pred=logreg.predict(X_val)

acc_logreg = round(accuracy_score(y_pred,y_val)*100, 2)
acc_logreg

In [ ]:
submission = pd.DataFrame()

#copy the PassengerID from the test Data

submission["id"] = test_data["id"]
submission

In [ ]:
test_data = test_data.drop("id" , axis=1)
test_data.head()

In [ ]:
predictions = logreg.predict(test_data)
submission["Response"] = predictions
submission.to_csv("submission.csv", index=False)